## 所有参数变量设置

In [2]:
import dv_processing as dv
import torch
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
import os
from matplotlib import pyplot as plt
import mmap
import aiofiles
import sys
from datetime import datetime
sys.path.append('./k_search_funciton')  # 添加模块路径
import k_search_funciton.file_read as file_read
import k_search_funciton.dvs_generate as dvs_generate
import k_search_funciton.tag_detector as tag_detector
import k_search_funciton.event_filter as event_filter
import cv2
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import math
# 设置中文字体
try:
    plt.rcParams['font.sans-serif'] = ['WenQuanYi Micro Hei'] # 或者 'SimHei', 'Microsoft YaHei' 等
    plt.rcParams['axes.unicode_minus'] = False  # 避免负号显示为方块
except Exception as e:
    print(f"无法设置中文字体: {e}。 图表中的中文可能无法正常显示。")


INPUT_FOLDER = '/mnt/f/raw2event/CIFAR10_DVS_Better/CIFAR10_DVS_Better'  # 替换为你的文件夹路径
FILE_SUFFIX = "200521"  # 替换为你的文件尾号

# 查找匹配的文件
try:
    files = file_read.find_matching_files(INPUT_FOLDER, FILE_SUFFIX)        
except FileNotFoundError as e:
    print(f"错误：{str(e)}")

#读取pi数据时用的宽高
PI_IMAGE_WEIGHT = 692
PI_IMAGE_HEIGHT = 520

#生成模拟事件数据时用的k值
k_values_raw = [0.00018 * 29250, 20, 0.0001, 1e-7, 5e-9, 0.00001] #用于生成raw模拟事件数据（默认为dvs原始k）
k_values_rgb = [0.00018 * 29250, 20, 0.0001, 1e-7, 5e-9, 0.00001] #用于生成rgb模拟事件数据（默认为dvs原始k）

#tag检测轨迹用的比例
TAG_REF_WIDTH = 287      # AprilTag参考宽度（像素）
BARBARA_REF_SIZE = 861   # Barbara参考边长（像素）
BARBARA_GAP = 82         # Barbara与tag之间的间隔（像素）
margin_ratio = 0.03

#线程数量
N_WORKERS = 4 

#滤取事件数据的批量大小
BATCH_SIZE_FOR_EVENT = 100000

未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件
未找到521文件


## 运行一次

In [ ]:
############################################################
#                      数据加载部分                         #
############################################################
dv_frames, dv_frames_timestamps = file_read.load_frames(files['dv'])
dv_events_tensor = file_read.load_events(files['dv'])
rgb_frames = file_read.read_rgb_frames(files['rgb_frames'], PI_IMAGE_HEIGHT, PI_IMAGE_WEIGHT)
raw_frames = file_read.read_raw_frames(files['raw_frames'], PI_IMAGE_HEIGHT, PI_IMAGE_WEIGHT)
pi_timestamps, real_timestamps = file_read.read_metadata(files['metadata'])

############################################################
#                      生成模拟事件数据                      #
############################################################
# 单线程生成RGB事件
rgb_events_tensor = dvs_generate.generate_events_tensor(
pi_timestamps,  # PI相机时间戳
rgb_frames,     # RGB帧数据
is_rgb=True,    # 标记为RGB数据
k_values=k_values_rgb
)
# 单线程生成RAW事件
raw_events_tensor = dvs_generate.generate_events_tensor(
pi_timestamps,  # PI相机时间戳
raw_frames,     # RAW帧数据
is_rgb=False,   # 标记为非RGB (即RAW) 数据
k_values=k_values_raw
)

############################################################
#                      时间对齐                            #
############################################################
# 计算时间偏移量
time_offset = file_read.calculate_time_offset(pi_timestamps, real_timestamps)
# 调整DV帧时间戳
dv_frames_timestamps = dv_frames_timestamps - time_offset
# 调整DV事件时间戳
dv_events_tensor[:, 0] = dv_events_tensor[:, 0] - time_offset


############################################################
#                      轨迹数据（并行）                            #
############################################################
# 构造参数列表
margin_ratios = [margin_ratio] * N_WORKERS
tag_ref_widths = [TAG_REF_WIDTH] * N_WORKERS
barbara_ref_sizes = [BARBARA_REF_SIZE] * N_WORKERS
barbara_gaps = [BARBARA_GAP] * N_WORKERS
is_raws_rgb = [False] * N_WORKERS
is_raws_raw = [True] * N_WORKERS

rgb_frame_batches = tag_detector.split_batches(rgb_frames.numpy(), N_WORKERS)
raw_frame_batches = tag_detector.split_batches(raw_frames.numpy(), N_WORKERS)
ts_batches = tag_detector.split_batches(pi_timestamps, N_WORKERS)

dv_frame_batches = tag_detector.split_batches(dv_frames.numpy(), N_WORKERS)
dv_ts_batches = tag_detector.split_batches(dv_frames_timestamps, N_WORKERS)

#批量得到rgb帧裁剪框信息
# 多线程批量处理并显示进度
with ThreadPoolExecutor(max_workers=N_WORKERS) as executor:
    rgb_all_results = []
    for batch_result in tqdm(
        executor.map(
            tag_detector.process_batch,
            rgb_frame_batches, ts_batches,
            margin_ratios, tag_ref_widths, barbara_ref_sizes, barbara_gaps, is_raws_rgb
        ),
        total=N_WORKERS
    ):
        rgb_all_results.append(batch_result)

rgb_crops_info = [item for batch in rgb_all_results for item in batch]# 合并所有结果
rgb_crops_info.sort(key=lambda x: x[2])  # 按时间戳排序


#批量得到raw帧裁剪框信息
# 多线程批量处理并显示进度
with ThreadPoolExecutor(max_workers=N_WORKERS) as executor:
    raw_all_results = []
    for batch_result in tqdm(
        executor.map(
            tag_detector.process_batch,
            raw_frame_batches, ts_batches,
            margin_ratios, tag_ref_widths, barbara_ref_sizes, barbara_gaps, is_raws_raw
        ),
        total=N_WORKERS
    ):
        raw_all_results.append(batch_result)

raw_crops_info = [item for batch in raw_all_results for item in batch]# 合并所有结果
raw_crops_info.sort(key=lambda x: x[2])  #按时间戳排序

#批量得到dv帧裁剪框信息
# 多线程批量处理并显示进度
with ThreadPoolExecutor(max_workers=N_WORKERS) as executor:
    dv_all_results = []
    for batch_result in tqdm(
        executor.map(
            tag_detector.process_batch,
            dv_frame_batches, dv_ts_batches,
            margin_ratios, tag_ref_widths, barbara_ref_sizes, barbara_gaps, is_raws_rgb
        ),
        total=N_WORKERS
    ):
        dv_all_results.append(batch_result)

dv_crops_info = [item for batch in dv_all_results for item in batch]# 合并所有结果
dv_crops_info.sort(key=lambda x: x[2])  # x[2]是timestamp# 按时间戳排序
# 第0列: barbara_info - 包含检测到的Barbara标签信息，如多边形顶点、旋转角度、中心点等
# 第1列: tag_info - 包含检测到的AprilTag标签信息，如ID、位置等
# 第2列: timestamp - 当前帧的时间戳


############################################################
#                      滤取事件数据                         #
############################################################
# 假设 box_size 是一个元组 (w, h)
def round_up_to_10(x):
    return int(math.ceil(x / 10.0) * 10)

# 往大取十的整数倍 作为滤取事件数据的最大边长
RGB_BOX_SIZE_FOR_EVENT = round_up_to_10(max(rgb_crops_info[0][0]['polygon'].ptp(axis=0)))
RAW_BOX_SIZE_FOR_EVENT = round_up_to_10(max(raw_crops_info[0][0]['polygon'].ptp(axis=0)))
DV_BOX_SIZE_FOR_EVENT = round_up_to_10(max(dv_crops_info[0][0]['polygon'].ptp(axis=0)))

#RGB事件滤取
# 使用并行处理函数
filtered_events_rgb = event_filter.filter_events_parallel(
    events_tensor=rgb_events_tensor,  # 你的事件数据
    crops_info=rgb_crops_info,        # 裁剪框信息
    target_size=RGB_BOX_SIZE_FOR_EVENT,  # 目标裁剪框大小
    transform=True,                    # 是否变换坐标
    batch_size=BATCH_SIZE_FOR_EVENT,                # 每批处理的事件数量
    n_workers=N_WORKERS                       # 并行进程数
)

#RAW事件滤取
# 使用并行处理函数
filtered_events_raw = event_filter.filter_events_parallel(
    events_tensor=raw_events_tensor,  # 你的事件数据
    crops_info=raw_crops_info,        # 裁剪框信息
    target_size=RAW_BOX_SIZE_FOR_EVENT,  # 目标裁剪框大小
    transform=True,                    # 是否变换坐标
    batch_size=BATCH_SIZE_FOR_EVENT,                # 每批处理的事件数量
    n_workers=N_WORKERS                       # 并行进程数
)

#DV事件滤取
# 使用并行处理函数
filtered_events_dv = event_filter.filter_events_parallel(
    events_tensor=dv_events_tensor,  # 你的事件数据
    crops_info=dv_crops_info,        # 裁剪框信息
    target_size=DV_BOX_SIZE_FOR_EVENT,  # 目标裁剪框大小
    transform=True,                    # 是否变换坐标
    batch_size=BATCH_SIZE_FOR_EVENT,                # 每批处理的事件数量
    n_workers=N_WORKERS                       # 并行进程数
)


############################################################
#                      结束                              #
############################################################

## 读取可视化

In [ ]:
# 创建一个包含两个子图的图形
plt.figure(figsize=(15, 6))  # 调整总图形大小

# 第一个子图：RAW帧
plt.subplot(1, 2, 1)  # 1行2列的第1个
plt.imshow(raw_frames[0].numpy(), cmap='gray')
plt.title('RAW Frame 0')
plt.axis('off')  # 关闭坐标轴

# 第二个子图：RGB帧
plt.subplot(1, 2, 2)  # 1行2列的第2个
plt.imshow(rgb_frames[0].numpy())
plt.title('RGB Frame 0')
plt.axis('off')  # 关闭坐标轴

# 调整子图之间的间距
plt.tight_layout()

# 显示图形
plt.show()

## 时间间隔直方图

In [ ]:
import k_search_funciton.interval_fit as interval_fit
import importlib
importlib.reload(interval_fit)


num_pixels_rgb, dt_rgb, mu_rgb, sigma_rgb = interval_fit.analyze_per_pixel_event_intervals_combined(
    events=filtered_events_rgb,
    min_events_per_pixel=10,         # 例如，每个像素至少需要10个事件
    max_dt_us_for_plot=100000,          # 例如，绘图时只看200微秒以下的间隔
    plot_bins=100,
    type='RGB'
)
plt.show()  # 显示图形

num_pixels_raw, dt_raw, mu_raw, sigma_raw = interval_fit.analyze_per_pixel_event_intervals_combined(
    events=filtered_events_raw,
    min_events_per_pixel=10,         # 例如，每个像素至少需要10个事件
    max_dt_us_for_plot=100000,          # 例如，绘图时只看200微秒以下的间隔
    plot_bins=100,
    type='RAW'
)
plt.show()  # 显示图形


num_pixels_dv, dt_dv, mu_dv, sigma_dv = interval_fit.analyze_per_pixel_event_intervals_combined(
    events=filtered_events_dv,
    min_events_per_pixel=10,         # 例如，每个像素至少需要10个事件
    max_dt_us_for_plot=100000,          # 例如，绘图时只看200微秒以下的间隔
    plot_bins=100,
    type='DV'
)
plt.show()  # 显示图形

import k_search_funciton.interval_fit as interval_fit
import importlib
importlib.reload(interval_fit)

# 频谱（时间间隔）
results = interval_fit.analyze_event_frequency_spectrum(
    filtered_events_raw,          # 事件数据
    max_freq_hz=100,              # 最大频率限制
    bins=50                       # 直方图区间数
)
plt.show()


# FFT频谱
fft_results = interval_fit.analyze_event_fft_spectrum(
    filtered_events_dv,
    sampling_rate=1000,   # 采样率 (Hz)
    max_freq_hz=100       # 最大频率限制
)
plt.show()


In [ ]:
print(len(filtered_events_dv))

In [ ]:
import importlib
importlib.reload(interval_fit)

# FFT频谱
fft_results = interval_fit.analyze_event_fft_spectrum(
    filtered_events_dv,
    sampling_rate=1000,   # 采样率 (Hz)
    max_freq_hz=100       # 最大频率限制
)
plt.show()

## 裁剪框可视化

In [ ]:
# 创建检测器
detector = tag_detector.create_detector()

# 处理RGB帧
barbara_info_rgb, cropped_rgb, ts_rgb = tag_detector.process_frame(
    rgb_frames[0].numpy(), pi_timestamps[0], detector, 
    margin_ratio=margin_ratio, 
    tag_ref_width=TAG_REF_WIDTH, 
    barbara_ref_size=BARBARA_REF_SIZE, 
    barbara_gap=BARBARA_GAP,
    is_raw=False
)

# 处理RAW帧
barbara_info_raw, cropped_raw, ts_raw = tag_detector.process_frame(
    raw_frames[0].numpy(), pi_timestamps[0], detector, 
    margin_ratio=margin_ratio, 
    tag_ref_width=TAG_REF_WIDTH, 
    barbara_ref_size=BARBARA_REF_SIZE, 
    barbara_gap=BARBARA_GAP,
    is_raw=True
)

# 处理DV帧
barbara_info_dv, cropped_dv, ts_dv = tag_detector.process_frame(
    dv_frames[0].numpy(), dv_frames_timestamps[0], detector, 
    margin_ratio=margin_ratio, 
    tag_ref_width=TAG_REF_WIDTH, 
    barbara_ref_size=BARBARA_REF_SIZE, 
    barbara_gap=BARBARA_GAP,
    is_raw=False
)

# 在展示图像之前添加以下三行
def get_box_size(polygon):
    if polygon is None:
        return None
    width = np.linalg.norm(polygon[1] - polygon[0])  # 底边长度
    height = np.linalg.norm(polygon[2] - polygon[1])  # 右边长度
    return (width, height)

print(f"RGB框大小: {get_box_size(barbara_info_rgb['polygon'] if barbara_info_rgb else None)}")
print(f"RAW框大小: {get_box_size(barbara_info_raw['polygon'] if barbara_info_raw else None)}")
print(f"DV框大小: {get_box_size(barbara_info_dv['polygon'] if barbara_info_dv else None)}")


#展示单帧裁剪效果
plt.figure(figsize=(12, 12))
# 1. RGB原图
plt.subplot(3, 2, 1)
frame_img_rgb = rgb_frames[0].numpy().copy()
plt.imshow(frame_img_rgb)
if barbara_info_rgb is not None:
    poly = barbara_info_rgb['polygon']
    x, y = poly[:,0], poly[:,1]
    plt.plot(x, y, '-', color='lime', linewidth=4)
    plt.plot([x[-1], x[0]], [y[-1], y[0]], '-', color='lime', linewidth=4)
plt.title('RGB Frame with Barbara Region')
plt.axis('off')

# 2. RGB裁剪
plt.subplot(3, 2, 2)
if cropped_rgb is not None:
    plt.imshow(cropped_rgb)
    plt.title('RGB Cropped Frame')
else:
    plt.title('RGB Cropped Frame (None)')
plt.axis('off')

# 3. RAW原图
plt.subplot(3, 2, 3)
frame_img_raw = raw_frames[0].numpy().copy()
plt.imshow(frame_img_raw, cmap='gray')
if barbara_info_raw is not None:
    poly = barbara_info_raw['polygon']
    x, y = poly[:,0], poly[:,1]
    plt.plot(x, y, '-', color='lime', linewidth=4)
    plt.plot([x[-1], x[0]], [y[-1], y[0]], '-', color='lime', linewidth=4)
plt.title('RAW Frame with Barbara Region')
plt.axis('off')

# 4. RAW裁剪
plt.subplot(3, 2, 4)
if cropped_raw is not None:
    plt.imshow(cropped_raw, cmap='gray')
    plt.title('RAW Cropped Frame')
else:
    plt.title('RAW Cropped Frame (None)')
plt.axis('off')

# 5. DV原图
plt.subplot(3, 2, 5)
frame_img_dv = dv_frames[0].numpy().copy()
plt.imshow(frame_img_dv, cmap='gray')
if barbara_info_dv is not None:
    poly = barbara_info_dv['polygon']
    x, y = poly[:,0], poly[:,1]
    plt.plot(x, y, '-', color='lime', linewidth=4)
    plt.plot([x[-1], x[0]], [y[-1], y[0]], '-', color='lime', linewidth=4)
plt.title('DV Frame with Barbara Region')
plt.axis('off')

# 6. DV裁剪
plt.subplot(3, 2, 6)
if cropped_dv is not None:
    plt.imshow(cropped_dv, cmap='gray')
    plt.title('DV Cropped Frame')
else:
    plt.title('DV Cropped Frame (None)')
plt.axis('off')

plt.tight_layout()
plt.subplots_adjust(wspace=0.05, hspace=0.15)  # 调小列间距
plt.show()

## 轨迹可视化

In [ ]:
plt.figure(figsize=(15, 10))

# 提取中心点轨迹
rgb_centers = np.array([info[0]['center'] for info in rgb_crops_info if info[0] is not None])
raw_centers = np.array([info[0]['center'] for info in raw_crops_info if info[0] is not None])
dv_centers  = np.array([info[0]['center'] for info in dv_crops_info  if info[0] is not None])


# 提取角度
rgb_angles = np.array([info[0]['angle'] for info in rgb_crops_info if info[0] is not None])
raw_angles = np.array([info[0]['angle'] for info in raw_crops_info if info[0] is not None])
dv_angles  = np.array([info[0]['angle'] for info in dv_crops_info  if info[0] is not None])

rgb_times = [info[2] for info in rgb_crops_info if info[0] is not None]
raw_times = [info[2] for info in raw_crops_info if info[0] is not None]
dv_times  = [info[2] for info in dv_crops_info  if info[0] is not None]

plt.figure(figsize=(10, 8))

# 1. 轨迹合并到一个图
plt.subplot(2, 1, 1)
s =1 # 点的大小
lw = 1  # 线宽

if len(rgb_centers) > 0:
    plt.plot(rgb_centers[:,0], rgb_centers[:,1], '-', color='red', lw=lw, label='RGB')
    plt.scatter(rgb_centers[:,0], rgb_centers[:,1], color='red', s=s)

if len(raw_centers) > 0:
    plt.plot(raw_centers[:,0], raw_centers[:,1], '-', color='green', lw=lw, label='RAW')
    plt.scatter(raw_centers[:,0], raw_centers[:,1], color='green', s=s)

if len(dv_centers) > 0:
    plt.plot(dv_centers[:,0], dv_centers[:,1], '-', color='blue', lw=lw, label='DV')
    plt.scatter(dv_centers[:,0], dv_centers[:,1], color='blue', s=s)

plt.title('Barbara Center Trajectories')
plt.xlabel('X')
plt.ylabel('Y')
plt.gca().invert_yaxis()
plt.axis('equal')
plt.grid(True)
plt.legend()

# 2. 角度随时间合并到一个图
plt.subplot(2, 1, 2)
s = 4  # 点的大小

if len(rgb_angles) > 0:
    plt.plot(rgb_times, rgb_angles, '-', color='red', lw=lw, label='RGB')
    plt.scatter(rgb_times, rgb_angles, color='red', s=s)

if len(raw_angles) > 0:
    plt.plot(raw_times, raw_angles, '-', color='green', lw=lw, label='RAW')
    plt.scatter(raw_times, raw_angles, color='green', s=s)

if len(dv_angles) > 0:
    plt.plot(dv_times, dv_angles, '-', color='blue', lw=lw, label='DV')
    plt.scatter(dv_times, dv_angles, color='blue', s=s)

plt.title('Barbara Angle vs Time')
plt.xlabel('Timestamp')
plt.ylabel('Angle (deg)')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

## 滤取事件数据播放可视化

In [ ]:
from IPython.display import display, clear_output
import datetime 

def play_filtered_events(events, interval_ms=33, window_size=(230, 230)):
    """
    播放过滤后的事件数据
    
    参数:
    events: 过滤后的事件数据 (PyTorch tensor)
    interval_ms: 每帧之间的时间间隔(毫秒)
    window_size: 显示窗口大小
    """
    # 创建事件累积器
    accumulator = dv.Accumulator(window_size)
    
    # 设置累积器参数
    accumulator.setEventContribution(0.25)
    accumulator.setNeutralPotential(0.5)
    accumulator.setMinPotential(0.0)
    accumulator.setMaxPotential(1.0)
    accumulator.setDecayFunction(dv.Accumulator.Decay.LINEAR)
    accumulator.setDecayParam(1e-6)
    accumulator.setSynchronousDecay(False)
    accumulator.setIgnorePolarity(False)
    
    # 创建预览窗口
    cv2.namedWindow("Events Preview", cv2.WINDOW_NORMAL)
    
    # 创建事件切片器
    slicer = dv.EventStreamSlicer()
    
    # 帧计数器
    frame_counter = 0
    
    def accumulate_events(event_slice):
        nonlocal frame_counter
        
        # 将事件切片传递给累积器
        accumulator.accept(event_slice)
        
        # 生成帧
        frame = accumulator.generateFrame()
        
        # 增加帧计数
        frame_counter += 1
        
        # 显示帧
        cv2.imshow("Events Preview", frame.image)
        cv2.waitKey(2)
    
    # 设置时间间隔
    slicer.doEveryTimeInterval(datetime.timedelta(milliseconds=interval_ms), accumulate_events)
    
    print("开始播放事件数据...")
    
    # 获取事件数据
    # 我们需要将PyTorch tensor转换为dv库可以处理的事件格式
    # 将事件分批处理，每批1000个事件
    batch_size = 1000
    total_events = len(events)
    
    for i in range(0, total_events, batch_size):
        # 获取当前批次的事件
        batch_events = events[i:min(i+batch_size, total_events)]
        
        # 转换为dv库可以处理的格式
        batch = dv.EventStore()
        for j in range(len(batch_events)):
            event = batch_events[j]
            x = int(event[0].item())
            y = int(event[1].item())
            timestamp = int(event[2].item())
            polarity = int(event[3].item())
            batch.push_back(dv.Event(x, y, timestamp, polarity))
        
        # 将事件传递给切片器
        slicer.accept(batch)
        
        # 检查是否按下'q'键退出
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    print(f"总帧数: {frame_counter}")
    print("事件播放完成")
    cv2.destroyAllWindows()


# 播放事件
play_filtered_events(filtered_events_dv, 3, (DV_BOX_SIZE_FOR_EVENT, DV_BOX_SIZE_FOR_EVENT))
# play_filtered_events(filtered_events_raw, 33, (RAW_BOX_SIZE_FOR_EVENT, RAW_BOX_SIZE_FOR_EVENT))
# play_filtered_events(filtered_events_rgb, 33, (RGB_BOX_SIZE_FOR_EVENT, RGB_BOX_SIZE_FOR_EVENT))


## 根据EMD黑盒搜索k

In [ ]:
#黑盒搜索拟合EMD

import csv  # 用于读写CSV文件
import os   # 用于处理文件路径和目录
import time # 用于记录计算时间
import torch # PyTorch库
import numpy as np # NumPy库
import optuna # Optuna优化库
from geomloss import SamplesLoss # 用于计算Sinkhorn距离 (EMD)


# 假设以下模块已经可以导入，并且包含了必要的函数
import k_search_funciton.dvs_generate as dvs_generate # 自定义DVS事件生成模块
import k_search_funciton.event_filter as event_filter   # 自定义事件过滤模块

# -------------------------------------------------
# 全局配置和预加载数据 (请确保这些变量在运行前已定义)
# -------------------------------------------------
OUTPUT_PATH = '/mnt/f/raw2event/0513_1305/k_search_grid_output_optuna' # Optuna结果的总输出路径
os.makedirs(OUTPUT_PATH, exist_ok=True) # 创建总输出路径 (如果不存在)

TRIAL_NUM = 3

np.random.seed(42)
torch.manual_seed(42)

# 设备配置 (GPU或CPU)
if torch.cuda.is_available(): # 检查CUDA是否可用
    device = torch.device("cuda:0") # 使用第一个GPU
    print(f"Using GPU: {torch.cuda.get_device_name(0)}") # 打印GPU名称
else:
    print("Warning: CUDA not available. Running on CPU. EMD might be slow.") # CUDA不可用警告
    device = torch.device("cpu") # 使用CPU

# Sinkhorn损失 (EMD) 参数配置
emd_blur = 0.01 # EMD计算的模糊参数
emd_scaling = 0.9 # EMD计算的缩放参数
# 创建Sinkhorn损失函数实例
sinkhorn_loss_fn = SamplesLoss(loss="sinkhorn", p=2, blur=emd_blur, scaling=emd_scaling, backend="auto")

# CSV文件路径，用于保存所有试验的总结结果
summary_csv_path = os.path.join(OUTPUT_PATH, "optuna_k_search_summary.csv")

# 事件归一化辅助函数
def normalize_events(ev_tensor):
    """将事件数据 (t, x, y) 归一化到 [0, 1] 区间"""
    ev_norm = ev_tensor.clone() # 克隆输入张量以避免修改原始数据
    if ev_norm.numel() == 0 or ev_norm.shape[1] < 3: # 如果事件为空或列数不足
        return ev_norm # 直接返回
    for i in range(3):  # 遍历时间(t), x坐标, y坐标
        min_val = ev_norm[:, i].min() # 计算当前列的最小值
        max_val = ev_norm[:, i].max() # 计算当前列的最大值
        if max_val > min_val: # 如果最大值大于最小值 (避免除以零)
            ev_norm[:, i] = (ev_norm[:, i] - min_val) / (max_val - min_val) # 执行归一化
        else:
            ev_norm[:, i] = 0.0 # 如果所有值相同，则设为0
    return ev_norm # 返回归一化后的事件张量

# Optuna 的目标函数
def objective(trial):
    """
    Optuna的目标函数。
    它接收一个'trial'对象，使用它建议k参数，
    运行事件生成和EMD计算，然后返回EMD距离。
    """
    # 1. Optuna建议k参数值
    # 为k1到k6参数建议搜索范围，可以根据经验调整这些范围
    k1_val = trial.suggest_float("k1", 0.1, 10.0)       # k1参数，范围0.1到10.0
    k2_val = trial.suggest_float("k2", 5.0, 50.0)        # k2参数，范围5.0到50.0
    k3_val = trial.suggest_float("k3", 1e-5, 1e-3, log=True) # k3参数，对数尺度，范围1e-5到1e-3
    k4_val = trial.suggest_float("k4", 1e-8, 1e-6, log=True) # k4参数，对数尺度，范围1e-8到1e-6
    k5_val = trial.suggest_float("k5", 1e-10, 1e-8, log=True)# k5参数，对数尺度，范围1e-10到1e-8
    k6_val = trial.suggest_float("k6", 1e-6, 1e-4, log=True) # k6参数，对数尺度，范围1e-6到1e-4
    
    k_params = [k1_val, k2_val, k3_val, k4_val, k5_val, k6_val] # 将建议的k值组成列表
    print(f"\nTrial {trial.number}: Testing k_params = {k_params}") # 打印当前试验编号和k参数

    # # 为当前试验创建单独的输出子文件夹 (可选，如果需要保存每组k值的详细结果)
    # current_trial_output_path = os.path.join(OUTPUT_PATH, f"trial_{trial.number}")
    # os.makedirs(current_trial_output_path, exist_ok=True)

    # 2. 使用自定义K值生成DVS模拟事件数据 (RAW)
    try:
        # 生成RAW事件
        raw_events_tensor = dvs_generate.generate_events_tensor(
            pi_timestamps,  # PI相机时间戳 (需在全局定义)
            raw_frames,     # RAW帧数据 (需在全局定义)
            is_rgb=False,   # 标记为非RGB (即RAW) 数据
            k_values=k_params # 使用当前Optuna建议的k_params
        )
    except Exception as e:
        print(f"  Error during RAW event generation: {e}") # 打印事件生成错误
        return float('inf') # 返回一个很大的值，表示此次试验失败

    # 3. 事件滤取git a
    try:
        # RAW事件滤取
        filtered_events_raw = event_filter.filter_events_parallel(
            events_tensor=raw_events_tensor,                # 输入RAW事件张量
            crops_info=raw_crops_info,                      # RAW帧的裁剪信息 (需在全局定义)
            target_size=RAW_BOX_SIZE_FOR_EVENT,             # RAW裁剪的目标尺寸 (需在全局定义)
            transform=True,                                 # 执行坐标变换
            batch_size=BATCH_SIZE_FOR_EVENT,                # 每批处理的事件数量 (需在全局定义)
            n_workers=N_WORKERS                   # 并行处理的工作线程数 (需在全局定义)
        )

        # DV事件滤取 (dv_events_tensor是固定的GT，不依赖k_params重新生成)
        filtered_events_dv = event_filter.filter_events_parallel(
            events_tensor=dv_events_tensor,                 # 输入预加载的DV事件张量 (需在全局定义)
            crops_info=dv_crops_info,                       # DV帧的裁剪信息 (需在全局定义)
            target_size=DV_BOX_SIZE_FOR_EVENT,              # DV裁剪的目标尺寸 (需在全局定义)
            transform=True,                                 # 执行坐标变换
            batch_size=BATCH_SIZE_FOR_EVENT,                # 每批处理的事件数量
            n_workers=N_WORKERS                   # 并行处理的工作线程数
        )
    except Exception as e:
        print(f"  Error during event filtering: {e}") # 打印事件过滤错误
        return float('inf') # 返回一个很大的值

    # 4. 计算 EMD 距离
    emd_distance = float('nan')  # 初始化EMD距离为NaN
    emd_calc_time = 0.0          # 初始化EMD计算时间

    # 检查事件是否为空
    if filtered_events_raw is None or filtered_events_raw.shape[0] == 0 or \
       filtered_events_dv is None or filtered_events_dv.shape[0] == 0:
        print("  RAW or DV events are empty after filtering, cannot calculate EMD.") # 事件为空的提示
        emd_distance = float('inf') # 如果事件为空，EMD设为无穷大以惩罚此试验
    else:
        # 转换为PyTorch张量并归一化
        normalized_raw_events = normalize_events(torch.from_numpy(filtered_events_raw)) # RAW事件归一化
        normalized_dv_events = normalize_events(torch.from_numpy(filtered_events_dv))   # DV事件归一化

        # 再次检查归一化后事件点云是否有效且至少有3列(t,x,y)
        if normalized_raw_events.numel() > 0 and normalized_raw_events.shape[1] >= 3 and \
           normalized_dv_events.numel() > 0 and normalized_dv_events.shape[1] >= 3:

            # 取t,x,y三列并转为float32，移动到目标设备
            points_raw = normalized_raw_events[:, :3].to(dtype=torch.float32, device=device) # 处理RAW点云
            points_dv = normalized_dv_events[:, :3].to(dtype=torch.float32, device=device)   # 处理DV点云

            # 确保点数大于0
            if points_raw.shape[0] > 0 and points_dv.shape[0] > 0:
                print(f"  Calculating EMD: RAW points {points_raw.shape[0]}, DV points {points_dv.shape[0]}") # 打印点数信息
                start_time = time.time()  # 记录开始时间
                try:
                    # 构造均匀权重
                    w_raw = torch.ones(points_raw.shape[0], device=device) / points_raw.shape[0] # RAW点云权重
                    w_dv = torch.ones(points_dv.shape[0], device=device) / points_dv.shape[0]   # DV点云权重
                    # 计算Sinkhorn距离
                    emd = sinkhorn_loss_fn(w_raw, points_raw, w_dv, points_dv) # 计算EMD
                    emd_distance = emd.item()  # 提取标量值
                    emd_calc_time = time.time() - start_time  # 计算耗时
                    print(f"  Sinkhorn-EMD: {emd_distance:.6f}, Time: {emd_calc_time:.2f}s") # 打印EMD结果和耗时
                except RuntimeError as e: # 捕获运行时错误
                    if "out of memory" in str(e).lower(): # 检查是否是显存不足错误
                        print("  CUDA out of memory during EMD calculation.") # 打印显存不足信息
                        emd_distance = float('inf') # 显存不足时，EMD设为无穷大
                    else:
                        print(f"  Runtime error during EMD calculation: {e}") # 其他运行时错误
                        emd_distance = float('inf') # 其他错误时，EMD设为无穷大
                except Exception as e: # 捕获其他未知错误
                    print(f"  Unknown error during EMD calculation: {e}") # 打印未知错误信息
                    emd_distance = float('inf') # 未知错误时，EMD设为无穷大
            else:
                print("  RAW or DV points are empty after pre-processing for EMD.") # 点云为空的提示
                emd_distance = float('inf') # 点云为空时，EMD设为无穷大
        else:
            print("  Normalized RAW or DV events are empty or malformed for EMD.") # 归一化后事件格式错误的提示
            emd_distance = float('inf') # 格式错误时，EMD设为无穷大


    return emd_distance # 返回EMD距离作为Optuna的优化目标

# -------------------------------------------------
# 主程序：运行Optuna研究
# -------------------------------------------------
# 检查关键全局变量是否已定义 (示例性检查，请根据实际情况补充)
required_globals = ['pi_timestamps', 'raw_frames', 'dv_events_tensor', 'raw_crops_info', 'dv_crops_info',
                    'RAW_BOX_SIZE_FOR_EVENT', 'DV_BOX_SIZE_FOR_EVENT', 'BATCH_SIZE_FOR_EVENT', 'N_WORKERS_FOR_EVENT']
for var_name in required_globals:
    if var_name not in globals():
        print(f"错误: 全局变量 '{var_name}' 未定义。请在运行Optuna优化前加载它。")
        exit()

print(f"Optuna study K值搜索开始。结果将保存到: {OUTPUT_PATH}") # 打印开始信息
print(f"所有试验的总结将记录在: {summary_csv_path}") # 打印总结文件路径

# 创建Optuna研究对象，目标是最小化objective函数的返回值 (EMD距离)
# 可以为 study_name 和 storage 指定数据库URL，以便持久化和恢复研究
# 例如 storage="sqlite:///example.db"
study = optuna.create_study(
    direction="minimize",
    study_name="dvs_k_param_optimization",
    storage="sqlite:///optuna_k_search.db",
    load_if_exists=True
)

# timeout参数可以用来限制总优化时间（秒）
study.optimize(objective, n_trials=TRIAL_NUM, timeout=None) # 运行100次试验
study.trials_dataframe().to_csv(summary_csv_path, index=False)
print(f"\n所有试验的详细结果已保存到: {summary_csv_path}")


# 优化结束后打印研究结果
print("\nOptuna K值搜索完成。") # 打印完成信息
print(f"最佳试验 Trial {study.best_trial.number}:") # 打印最佳试验编号
print(f"  最佳 EMD 值: {study.best_value:.6f}") # 打印最佳EMD值
print("  最佳参数 K:") # 打印最佳参数
for key, value in study.best_params.items(): # 遍历最佳参数
    print(f"    {key}: {value:.7g}") # 打印每个最佳参数值

print(f"\n所有试验的详细结果已保存到: {summary_csv_path}") # 提示结果保存路径

## 根据u和sigma黑盒搜索k

In [ ]:
# 黑盒搜索拟合u和sigma (使用Optuna)
# 确保运行了之前的 1, 2, 3, 4号代码块以加载必要数据和函数

import csv                          # CSV操作 (Optuna后可移除部分手动CSV操作)
import os                           # 操作系统功能，如路径处理
import numpy as np                  # Numpy库，用于数值计算
import matplotlib.pyplot as plt     # Matplotlib库，用于绘图
import optuna                       # Optuna优化库

# 确保以下自定义模块已加载且路径正确
import k_search_funciton.event_filter as event_filter
import k_search_funciton.dvs_generate as dvs_generate # 假设此模块存在
import k_search_funciton.interval_fit as interval_fit
import importlib
importlib.reload(event_filter)
importlib.reload(dvs_generate) # 如果有修改，重新加载
importlib.reload(interval_fit) # 如果有修改，重新加载
import math

# 假设 box_size 是一个元组 (w, h)
def round_up_to_10(x):
    return int(math.ceil(x / 10.0) * 10)

# 取每个框的最大边长
rgb_box_size = max(barbara_info_rgb['polygon'].ptp(axis=0))
raw_box_size = max(barbara_info_raw['polygon'].ptp(axis=0))
dv_box_size = max(barbara_info_dv['polygon'].ptp(axis=0))

# 往大取十的整数倍
RGB_BOX_SIZE_FOR_EVENT = round_up_to_10(rgb_box_size)
RAW_BOX_SIZE_FOR_EVENT = round_up_to_10(raw_box_size)
DV_BOX_SIZE_FOR_EVENT = round_up_to_10(dv_box_size)
BATCH_SIZE_FOR_EVENT = 100000
N_WORKERS_FOR_EVENT = 4


TRIAL_NUM = 3

# --- 配置和预加载数据 (应从之前的代码块继承) ---
OUTPUT_PATH = '/mnt/f/raw2event/0513_1305/k_search_optuna_output' # Optuna结果的新输出路径
os.makedirs(OUTPUT_PATH, exist_ok=True) # 创建输出路径

# 初始K值 (作为Optuna搜索的中心参考)
k1_initial = 5.0
k2_initial = 20.0
k3_initial = 0.0001
k4_initial = 1e-7
k5_initial = 5e-9
k6_initial = 1e-5

# --- DV事件处理 (只需执行一次，在Optuna优化外部) ---
print("正在处理DV事件 (仅一次)...")
# DV事件滤取
filtered_events_dv = event_filter.filter_events_parallel(
    events_tensor=dv_events_tensor,           # 输入预加载的DV事件张量
    crops_info=dv_crops_info,                 # DV帧的裁剪信息
    target_size=DV_BOX_SIZE_FOR_EVENT,        # DV裁剪的目标尺寸
    transform=True,                           # 执行坐标变换
    batch_size=BATCH_SIZE_FOR_EVENT,          # 每批处理的事件数量
    n_workers=N_WORKERS_FOR_EVENT             # 并行处理的工作线程数
)

# 对滤取后的DV事件进行间隔拟合
num_pixels_dv_global, dt_dv_global, mu_dv_global, sigma_dv_global = interval_fit.analyze_per_pixel_event_intervals_combined(
    events=filtered_events_dv,          # 输入滤取后的DV事件
    min_events_per_pixel=10,            # 每个像素最少事件数
    max_dt_us_for_plot=100000,          # 绘图显示的最大时间间隔
    plot_bins=100,                      # 直方图的bins数量
    type='DV_Global'                    # 数据类型标记
)
# 保存DV拟合的图像和数据 (仅一次)
if mu_dv_global is not np.nan : # 检查拟合是否成功
    plt.savefig(os.path.join(OUTPUT_PATH, 'interval_fit_dv_global.png'))
    plt.close()
    np.savez(os.path.join(OUTPUT_PATH, 'interval_fit_dv_global.npz'),
             num_pixels=num_pixels_dv_global, dt=dt_dv_global, mu=mu_dv_global, sigma=sigma_dv_global)
    print(f"DV事件拟合完成: mu_dv={mu_dv_global:.4f}, sigma_dv={sigma_dv_global:.4f}")
else:
    print("错误：DV事件拟合失败，无法进行优化。请检查DV数据和拟合函数。")
    # 根据情况决定是否退出 exit()

# --- Optuna 目标函数定义 ---
def objective(trial):
    global mu_dv_global, sigma_dv_global # 确保能访问全局的DV拟合结果

    # 1. Optuna建议K参数值 (对数尺度搜索)
    # 调整搜索范围因子，例如k_initial/factor 到 k_initial*factor
    # 对较小值使用更大的因子范围，如100x；对较大值使用较小因子，如10x
    k1_val = trial.suggest_float("k1", k1_initial / 10, k1_initial * 10, log=True)
    k2_val = trial.suggest_float("k2", k2_initial / 10, k2_initial * 10, log=True)
    k3_val = trial.suggest_float("k3", k3_initial / 100, k3_initial * 100, log=True)
    k4_val = trial.suggest_float("k4", k4_initial / 100, k4_initial * 100, log=True)
    k5_val = trial.suggest_float("k5", k5_initial / 100, k5_initial * 100, log=True)
    k6_val = trial.suggest_float("k6", k6_initial / 100, k6_initial * 100, log=True)
    
    k_params = [k1_val, k2_val, k3_val, k4_val, k5_val, k6_val]
    print(f"\nTrial {trial.number}: 测试 K参数 = {k_params}")

    # --- 2. 使用建议的K值生成DVS模拟事件数据 ---
    try:
        raw_events_tensor = dvs_generate.generate_events_tensor(
            pi_timestamps,  # PI相机时间戳 (需全局可用)
            raw_frames,     # RAW帧数据 (需全局可用)
            is_rgb=False,   # 标记为非RGB (即RAW) 数据
            k_values=k_params
        )
    except Exception as e:
        print(f"  错误：RAW事件生成失败: {e}")
        return float('inf') # 事件生成失败，返回一个很大的损失值

    # --- 3. 事件滤取 (RAW) ---
    try:
        filtered_events_raw = event_filter.filter_events_parallel(
            events_tensor=raw_events_tensor,
            crops_info=raw_crops_info,             # RAW帧的裁剪信息 (需全局可用)
            target_size=RAW_BOX_SIZE_FOR_EVENT,    # RAW裁剪的目标尺寸 (需全局可用)
            transform=True,
            batch_size=BATCH_SIZE_FOR_EVENT,       # (需全局可用)
            n_workers=N_WORKERS                   # (需全局可用)
        )
    except Exception as e:
        print(f"  错误：RAW事件滤波失败: {e}")
        return float('inf') # 事件滤波失败

    # --- 4. 间隔拟合 (RAW) ---
    # 注意：analyze_per_pixel_event_intervals_combined 内部有绘图，Optuna大量迭代时应考虑关闭或修改
    # 为避免过多图像文件，这里的plt.savefig和plt.close可以注释掉，或只为最佳trial保存
    num_pixels_raw, dt_raw, mu_raw, sigma_raw = interval_fit.analyze_per_pixel_event_intervals_combined(
        events=filtered_events_raw,
        min_events_per_pixel=10,
        max_dt_us_for_plot=100000, # 在objective中可以设为更小的值或不绘图以加速
        plot_bins=100,
        type=f'RAW_Trial_{trial.number}' # 区分不同试验的类型名
    )
    plt.close() # 关闭interval_fit内部的绘图，避免显示过多窗口

    # 检查拟合结果是否有效
    if mu_raw is np.nan or sigma_raw is np.nan or mu_dv_global is np.nan or sigma_dv_global is np.nan:
        print(f"  警告：拟合参数包含NaN (mu_raw={mu_raw}, sigma_raw={sigma_raw})。此试验将被惩罚。")
        return float('inf') # 拟合失败或DV基准无效，返回很大的损失值

    print(f"  RAW拟合结果: mu_raw={mu_raw:.4f}, sigma_raw={sigma_raw:.4f}")
    
    # --- 5. 计算损失函数 ---
    # 目标是让mu_raw接近mu_dv, sigma_raw接近sigma_dv
    # 可以使用绝对差之和，或归一化的平方差之和等
    # 为避免除零，对分母加一个小数
    loss_mu = (abs(mu_raw - mu_dv_global)) / (abs(mu_dv_global) + 1e-9) 
    loss_sigma = (abs(sigma_raw - sigma_dv_global)) / (abs(sigma_dv_global) + 1e-9)
    
    # 可以给mu和sigma的匹配赋予不同权重，这里简单相加
    total_loss = loss_mu + loss_sigma 
    
    print(f"  损失值: {total_loss:.6f} (mu_loss={loss_mu:.4f}, sigma_loss={loss_sigma:.4f})")
    
    return total_loss

# --- Optuna 研究设置与执行 ---
if mu_dv_global is not np.nan and sigma_dv_global is not np.nan: # 确保DV基准有效
    print("\n开始Optuna K参数优化搜索...")
    study = optuna.create_study(
        direction="minimize",                     # 优化目标是最小化损失函数
        study_name="dvs_k_params_optimization_6k", # 研究名称
        storage=f"sqlite:///{os.path.join(OUTPUT_PATH, 'optuna_6k_study.db')}", # SQLite数据库保存研究结果
        load_if_exists=True                       # 如果数据库已存在，则加载之前的研究
    )

    # 设置试验次数，例如100次
    study.optimize(objective, n_trials=TRIAL_NUM, timeout=None) # timeout参数可以用来限制总优化时间（秒）

    # --- 输出优化结果 ---
    print("\nOptuna K参数优化完成。")
    print(f"最佳试验 Trial {study.best_trial.number}:")
    print(f"  最佳损失值: {study.best_value:.6f}")
    print("  最佳参数 K:")
    for key, value in study.best_params.items():
        print(f"    {key}: {value:.7g}") # 使用科学计数法或适当格式打印

    # 你可以在这里添加代码，使用最佳参数重新运行一次完整的流程，并保存所有相关的输出文件(事件、拟合图等)
    # 例如:
    # best_k_params = [study.best_params[f'k{i+1}'] for i in range(6)]
    # ... 然后运行生成、滤波、拟合，并保存到特定文件夹 ...

    # 将所有试验结果保存到CSV文件
    try:
        trials_df = study.trials_dataframe()
        trials_df.to_csv(os.path.join(OUTPUT_PATH, "optuna_6k_trials_summary.csv"), index=False)
        print(f"\n所有试验的详细结果已保存到: {os.path.join(OUTPUT_PATH, 'optuna_6k_trials_summary.csv')}")
    except Exception as e:
        print(f"保存Optuna试验总结到CSV时出错: {e}")
else:
    print("由于DV事件拟合失败，Optuna优化未启动。")

print("\n--- 脚本执行完毕 ---")
